# 1. Finnhub API 기반 뉴스 데이터 수집

### 1-1. 라이브러리 Import
필요한 Python 라이브러리들을 가져옵니다. 웹 API 호출, 데이터 처리, 날짜 처리, 환경변수 관리 등에 사용됩니다.

In [22]:
# 라이브러리 import
import requests           # HTTP 요청을 위한 라이브러리 (Finnhub API 호출용)
import pandas as pd       # 데이터 분석 및 처리를 위한 라이브러리 (DataFrame 사용)
import datetime           # 날짜 및 시간 처리를 위한 라이브러리
import time               # 시간 지연 처리를 위한 라이브러리 (API 호출 제한 관리)

### 1-2. 환경 설정 로드
Finnhub API 키를 로드합니다.

In [23]:
FINHUB_API_KEY = ("d16f6mpr01qvtdbi7280d16f6mpr01qvtdbi728g")

### 1-3. API 호출 제한 설정
Finnhub API는 분당 호출 횟수에 제한이 있습니다. API 호출 간격을 조절하여 제한을 위반하지 않도록 설정합니다.

In [24]:
# API 호출 제한 설정
# Finnhub Free Tier는 분당 60회 호출 제한이 있음
API_CALLS_PER_MINUTE = 60

# 각 API 호출 간의 최소 대기 시간 계산 (초 단위)
# 60초 / 60회 = 1초 간격으로 호출하여 제한 내에서 안전하게 호출
DELAY_BETWEEN_CALLS = 60.0 / API_CALLS_PER_MINUTE

### 1-4. 안전한 datetime 변환 함수
Finnhub API에서 받은 timestamp를 pandas datetime으로 변환할 때 발생할 수 있는 오류를 방지하는 함수입니다.

In [25]:
# 안전한 datetime 변환 함수
def safe_datetime_conversion(timestamp):

    # 빈 값이나 0값 체크
    if not timestamp or timestamp == 0:
        return None
    
    try:
        # 유닉스 타임스탬프 범위 확인 (1970-01-01 이후만 허용)
        # 음수값은 1970년 이전을 의미하므로 제외
        if timestamp < 0:
            return None
            
        # pandas에서 처리 가능한 최대 timestamp 확인
        # 2262-04-11 이후는 pandas에서 overflow 발생
        if timestamp > 9223372036:  # 약 2262-04-11 23:47:16
            return None
            
        # Unix timestamp를 pandas datetime으로 변환
        # unit='s'는 초 단위임을 의미
        return pd.to_datetime(timestamp, unit='s')
        
    except (ValueError, OutOfBoundsDatetime, OverflowError):
        # pandas에서 발생하는 날짜 범위 오류 처리
        return None
    except Exception:
        # 기타 예상치 못한 오류 처리
        return None


### 1-5. Finnhub 뉴스 수집 함수 (메인 함수)
특정 기업의 뉴스를 대량으로 수집하는 핵심 함수입니다. 날짜 구간을 나누어 여러 번 API 호출하여 최대한 많은 뉴스를 수집합니다.

In [26]:
# Finnhub 뉴스 수집 함수 (대량 수집 최적화)
def fetch_finnhub_news_extended(symbol: str = "GOOGL", start_date: str = "2025-06-14", days_per_request: int = 30) -> pd.DataFrame:

    # API 키가 없으면 빈 DataFrame 반환
    if not FINHUB_API_KEY:
        return pd.DataFrame()

    # Finnhub Company News API 엔드포인트
    url = "https://finnhub.io/api/v1/company-news"
    
    try:
        # 현재 날짜 기준으로 수집 기간 설정
        today = datetime.date.today()
        
        # 3년 전부터 현재까지 수집 시도 (Free Tier 제한 테스트)
        three_years_ago = today - datetime.timedelta(days=1095)  # 1095일 = 약 3년
        actual_start = three_years_ago  # 실제 수집 시작 날짜
        actual_end = today             # 실제 수집 종료 날짜
        
    except ValueError:
        # 날짜 처리 오류시 빈 DataFrame 반환
        return pd.DataFrame()

    # 모든 수집된 기사를 저장할 리스트
    all_articles = []
    # 현재 처리 중인 날짜
    current_date = actual_start
    # API 호출 횟수 카운터
    request_count = 0
    
    # 시작 날짜부터 종료 날짜까지 반복 처리
    while current_date < actual_end:
        # 각 요청의 종료 날짜 계산 (지정된 일수만큼 또는 전체 종료 날짜까지)
        period_end = min(current_date + datetime.timedelta(days=days_per_request), actual_end)
        
        # API 요청 파라미터 설정
        params = {
            "symbol": symbol,                           # 주식 심볼
            "from": current_date.isoformat(),           # 시작 날짜 (YYYY-MM-DD)
            "to": period_end.isoformat(),               # 종료 날짜 (YYYY-MM-DD)
            "token": FINHUB_API_KEY                     # API 인증 토큰
        }
        
        try:
            request_count += 1  # API 호출 횟수 증가
            
            # API 호출 제한을 위한 딜레이 (분당 60회 제한 준수)
            time.sleep(DELAY_BETWEEN_CALLS)
            
            # HTTP GET 요청으로 뉴스 데이터 가져오기
            res = requests.get(url, params=params, timeout=30)
            
            # HTTP 429 (Too Many Requests) 오류 처리
            if res.status_code == 429:
                time.sleep(10)  # 10초 대기 후 재시도
                res = requests.get(url, params=params, timeout=30)
            
            # HTTP 403 (Forbidden) 오류시 수집 중단 (API 제한 도달)
            if res.status_code == 403:
                break
            
            # 기타 HTTP 오류 처리
            if res.status_code != 200:
                if res.status_code != 429:
                    # 429가 아닌 오류는 다음 기간으로 건너뛰기
                    current_date = period_end + datetime.timedelta(days=1)
                    continue
                else:
                    # 429 오류는 30초 대기 후 재시도
                    time.sleep(30)
                    continue

            # JSON 응답 데이터 파싱
            data = res.json()
            
            # 응답이 리스트가 아닌 경우 (오류 응답 등) 처리
            if not isinstance(data, list):
                if isinstance(data, dict) and 'error' in data:
                    # API 제한 관련 오류 메시지 확인
                    if 'limit' in data['error'].lower():
                        break  # 제한 도달시 수집 중단
                # 다음 기간으로 이동
                current_date = period_end + datetime.timedelta(days=1)
                continue

            # 해당 기간의 뉴스 데이터 처리
            period_articles = []
            for item in data:
                # API에서 받은 timestamp를 안전하게 datetime으로 변환
                pub_date = safe_datetime_conversion(item.get("datetime"))
                
                # 뉴스 기사 정보를 딕셔너리로 구성
                article = {
                    "id": item.get("id"),                           # 기사 고유 ID
                    "title": item.get("headline", ""),             # 기사 제목
                    "summary": item.get("summary", ""),            # 기사 요약
                    "link": item.get("url", ""),                   # 기사 URL
                    "publisher": item.get("publisher", ""),        # 발행사
                    "category": item.get("category", ""),          # 카테고리
                    "pubDate": pub_date,                           # 발행 날짜
                    "image": item.get("image", ""),                # 이미지 URL
                    "related": item.get("related", ""),            # 관련 정보
                    "source": item.get("source", ""),              # 소스
                    "collection_period": f"{current_date.isoformat()}_{period_end.isoformat()}"  # 수집 기간 정보
                }
                period_articles.append(article)
            
            # 현재 기간의 기사들을 전체 리스트에 추가
            all_articles.extend(period_articles)
            
        except Exception as e:
            # 예외 발생시 API 제한 관련 오류인지 확인
            if "limit" in str(e).lower() or "403" in str(e):
                break  # API 제한 관련 오류시 수집 중단
        
        # 다음 기간으로 이동 (하루 간격으로 겹치지 않게)
        current_date = period_end + datetime.timedelta(days=1)
        
        # API 호출 제한 방지를 위한 추가 대기
        # 10회 호출마다 2초 추가 대기로 안전성 확보
        if request_count % 10 == 0:
            time.sleep(2)

    # 수집된 모든 기사 데이터 처리
    if all_articles:
        # 리스트를 pandas DataFrame으로 변환
        df = pd.DataFrame(all_articles)
        
        # 중복 기사 제거 (ID, 제목, 링크 기준으로 중복 판별)
        df = df.drop_duplicates(subset=['id', 'title', 'link'])
        
        # 날짜순으로 정렬 (최신 기사가 위에 오도록)
        # na_position='last'로 날짜가 없는 기사는 맨 아래로
        df = df.sort_values('pubDate', ascending=False, na_position='last')
        
        return df
    else:
        # 수집된 기사가 없으면 빈 DataFrame 반환
        return pd.DataFrame()


### 1-6. 수집 설정값 정의 - 하드코딩 필요
뉴스 수집을 위한 기본 설정값들을 정의합니다. 필요에 따라 이 값들을 수정하여 다른 기업이나 기간의 뉴스를 수집할 수 있습니다.

In [27]:
# 뉴스 수집을 위한 설정값 정의
# 수집 대상 기업의 주식 심볼 (미국 상장 기업)
# 예시: "AAPL"(Apple), "GOOGL"(Google), "TSLA"(Tesla), "MSFT"(Microsoft), "AMZN"(Amazon) 등
target_symbol = "AAPL"

# 참고용 시작 날짜 (실제로는 3년 전부터 자동으로 수집됨)
# YYYY-MM-DD 형식으로 입력
start_date = "2025-06-14"

# 한 번의 API 호출당 수집할 일수 설정
# 값이 작을수록 더 많은 API 호출을 하게 되어 더 많은 뉴스를 수집할 수 있음
# 하지만 API 제한에 도달할 가능성도 높아짐
days_per_request = 7

### 1-7. 뉴스 데이터 수집 실행
위에서 정의한 설정값들을 사용하여 실제로 뉴스 데이터를 수집합니다. 이 과정에서 API 호출이 여러 번 발생하므로 시간이 걸릴 수 있습니다.


In [28]:
# 뉴스 데이터 수집 실행
# 위에서 정의한 함수와 설정값을 사용하여 뉴스 수집을 시작
# 이 과정은 API 호출 제한으로 인해 시간이 걸릴 수 있음 (몇 분에서 몇십 분)
df_extended_news = fetch_finnhub_news_extended(
    symbol=target_symbol,           # 수집할 기업의 주식 심볼
    start_date=start_date,          # 참고용 시작 날짜
    days_per_request=days_per_request  # 한 번에 수집할 일수
)

### 1-8. 수집 결과 확인 및 기본 분석
수집된 뉴스 데이터의 기본적인 통계 정보를 확인합니다. 총 기사 수, 수집 기간, 유효한 날짜 정보 등을 출력합니다.

In [ ]:
# 수집 결과 확인 및 기본 통계 분석
# DataFrame이 비어있지 않은지 확인 (수집에 성공했는지 체크)
if not df_extended_news.empty:
    # 총 수집된 기사 수 출력
    print(f"총 수집 기사 수: {len(df_extended_news)}개")
    
    # 날짜별 기사 분포 분석
    if 'pubDate' in df_extended_news.columns:
        # 유효한 날짜 정보가 있는 기사들만 필터링
        valid_dates = df_extended_news[df_extended_news['pubDate'].notna()].copy()
        
        if not valid_dates.empty:
            # 날짜만 추출 (시간 정보 제거)
            valid_dates['date_only'] = valid_dates['pubDate'].dt.date
            
            # 날짜별 기사 수 집계 및 정렬
            date_counts = valid_dates['date_only'].value_counts().sort_index()
            
            # 수집 기간 (가장 오래된 날짜 ~ 가장 최근 날짜) 출력
            print(f"수집 기간: {date_counts.index.min()} ~ {date_counts.index.max()}")
            
            # 유효한 날짜가 있는 기사 수와 전체 기사 수 비교
            print(f"유효한 날짜 기사: {len(valid_dates)}개 / 전체 {len(df_extended_news)}개")
else:
    # 수집에 실패한 경우 오류 메시지 출력
    print("뉴스 수집에 실패했습니다.")


### 1-9. 데이터 미리보기
수집된 뉴스 데이터의 일부를 표 형태로 확인합니다. 제목, 발행처, 발행 날짜 등 주요 정보를 미리 볼 수 있습니다.

In [ ]:
# 수집된 뉴스 데이터 미리보기
# 데이터가 정상적으로 수집되었는지 확인
if not df_extended_news.empty:
    print("최신 뉴스 미리보기:")
    
    # 주요 컬럼들만 선택하여 상위 5개 기사 표시
    # title: 기사 제목, summary: 본문 내용, pubDate: 발행 날짜
    preview_data = df_extended_news[['title', 'summary','pubDate']].head()
    
    # Jupyter Notebook에서 표 형태로 깔끔하게 출력
    display(preview_data)


### 1-10. CSV 파일로 저장
수집된 뉴스 데이터를 CSV 파일로 저장합니다. 파일명에는 기업 심볼과 현재 날짜가 포함되어 구분하기 쉽게 만듭니다.

In [ ]:
# 수집된 뉴스 데이터를 CSV 파일로 저장
# 데이터가 있는 경우에만 저장 진행
if not df_extended_news.empty:
    # 현재 날짜를 ISO 형식 (YYYY-MM-DD)으로 가져오기
    # today = datetime.date.today().isoformat()
    
    # 파일명 생성: "기업심볼_extended_news_날짜.csv" 형식
    # 예: "GOOGL_extended_news_2025-06-19.csv"
    filename = f"{target_symbol}_extended_news_{start_date}.csv"
    
    # DataFrame을 CSV 파일로 저장
    # index=False: 행 번호를 파일에 포함하지 않음
    # encoding='utf-8-sig': 한글 등 유니코드 문자가 깨지지 않도록 설정
    df_extended_news.to_csv(filename, index=False, encoding='utf-8-sig')
    
    # 저장 완료 메시지 출력
    print(f"파일 저장 완료: {filename}")
else:
    # 데이터가 없는 경우 경고 메시지 출력
    print("저장할 데이터가 없습니다.")


# 2. Finnhub API 기반 뉴스 데이터 전처리 및 FinBERT 감정분석

### 2-1. 파일 경로 설정
분석할 뉴스 데이터 파일의 읽기 경로와 감정분석 결과를 저장할 경로를 설정합니다.

In [2]:
# 입력 파일 경로: Finnhub에서 수집한 Apple 뉴스 데이터 CSV 파일
read_path = "../data/AAPL_extended_news_2025-06-14.csv"

# 출력 파일 경로: FinBERT 감정분석 결과가 추가된 CSV 파일
write_path = "../data/apple_finbert_finnhub.csv"

### 2-2. 데이터 로드
pandas를 사용하여 Finnhub에서 수집한 뉴스 데이터를 DataFrame으로 로드합니다.

In [3]:
# 데이터 분석을 위한 pandas 라이브러리 import
import pandas as pd

# CSV 파일에서 뉴스 데이터를 DataFrame으로 로드
# 파일에는 id, title, summary, link, publisher, pubDate 등의 컬럼이 포함됨
df = pd.read_csv(read_path)

### 2-3. FinBERT 모델 설정 및 데이터 전처리
금융 도메인에 특화된 FinBERT 모델을 로드하고, 뉴스 텍스트를 감정분석에 적합하게 전처리합니다.

### FinBERT 전처리를 최소한으로 유지한 이유

- 사전훈련 데이터: 수만 개의 금융 뉴스, 보고서, 소셜미디어 텍스트로 훈련
- 도메인 특화 어휘: 금융 용어, 표현, 맥락을 이미 학습
- 노이즈 처리 능력: 일반적인 텍스트 노이즈에 이미 강건함
- 컨텍스트 이해: 문맥을 통한 의미 파악
- 전이학습 효과: 대규모 사전훈련으로 일반화 능력 확보

In [ ]:
# 감정분석을 위한 transformers 라이브러리 import
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# FinBERT 모델 설정 - 금융 뉴스 감정분석에 특화된 사전훈련 모델
MODEL = "yiyanghkust/finbert-tone"  # Hugging Face의 FinBERT-tone 모델

# 토크나이저 로드 - 텍스트를 토큰으로 변환하는 도구
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# 사전훈련된 FinBERT 모델 로드 - 금융 텍스트의 감정을 분류하는 모델
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# 감정분석 파이프라인 생성
# return_all_scores=True: positive, neutral, negative 모든 점수를 반환
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

# 이전에 로드한 뉴스 데이터 사용
df = pd.read_csv(read_path)

# 빈 값(NaN)을 빈 문자열로 대체하여 처리 오류 방지
df = df.fillna("")

# 분석할 텍스트 컬럼 생성 - 뉴스 제목을 감정분석 대상으로 설정
df["text"] = df["title"] + df["summary"]

# 텍스트를 BERT 모델의 최대 토큰 길이에 맞게 자르고, 
# 원본이 길이 제한을 초과했는지 플래그로 표시하는 함수
def truncate_and_flag(text, tokenizer, max_len=512):
    
    # 텍스트를 토큰으로 변환하면서 max_len에 맞게 자르기
    tokens = tokenizer.encode(text, truncation=True, max_length=max_len)
    
    # 잘린 토큰을 다시 텍스트로 변환
    truncated_text = tokenizer.decode(tokens, skip_special_tokens=True)
    
    # 원본 텍스트가 최대 길이를 초과했는지 확인
    # 1: 초과함, 0: 초과하지 않음
    over_flag = 1 if len(tokenizer.encode(text)) > max_len else 0
    
    # 잘린_텍스트, 초과_플래그 반환
    return truncated_text, over_flag

# 모든 텍스트에 대해 길이 제한 적용 및 초과 플래그 생성
# apply()와 pd.Series()를 사용하여 함수 결과를 두 개의 컬럼으로 분할
df[["text", "over_512"]] = df["text"].apply(
    lambda x: pd.Series(truncate_and_flag(x, tokenizer, max_len=512))
)


### 2-4. FinBERT 감정분석 실행
전처리된 뉴스 텍스트에 대해 FinBERT 모델을 사용하여 감정분석을 수행하고 결과를 저장합니다.

In [ ]:
# FinBERT 감정분석 실행을 위한 텍스트 리스트 준비
# DataFrame의 text 컬럼을 파이썬 리스트로 변환
texts = df["text"].tolist()

# FinBERT 모델로 배치 감정분석 수행
# batch_size=8: 한 번에 8개씩 처리하여 메모리 효율성과 속도 최적화
# GPU 메모리 부족시 더 작은 값(예: 4, 2)으로 조정 가능
results = finbert(texts, batch_size=8)

# 감정분석 결과를 저장할 빈 리스트들 초기화
pos_scores, neu_scores, neg_scores = [], [], []

# 각 텍스트의 감정분석 결과를 순회하며 점수 추출
for res in results:
    # 결과를 라벨:점수 형태의 딕셔너리로 변환
    # 라벨을 소문자로 통일하여 일관성 확보
    d = {r["label"].lower(): r["score"] for r in res}
    
    # 각 감정에 대한 점수를 리스트에 추가 (없으면 0.0으로 기본값 설정)
    pos_scores.append(d.get("positive", 0.0))   # 긍정 감정 점수
    neu_scores.append(d.get("neutral", 0.0))    # 중립 감정 점수  
    neg_scores.append(d.get("negative", 0.0))   # 부정 감정 점수

# 감정분석 결과를 DataFrame에 새로운 컬럼으로 추가
df["finbert_positive"] = pos_scores   # FinBERT 긍정 점수 (0~1)
df["finbert_neutral"] = neu_scores    # FinBERT 중립 점수 (0~1)
df["finbert_negative"] = neg_scores   # FinBERT 부정 점수 (0~1)

# 감정분석 결과가 추가된 DataFrame을 CSV 파일로 저장
# index=False: 행 번호를 파일에 포함하지 않음
df.to_csv(write_path, index=False)

# 저장 완료 메시지 출력
print(f"결과 저장 완료: {write_path}")


# 3. Rapid API 기반 트위터(X) 데이터 수집

### 3-1. 라이브러리 Import
트윗 수집에 필요한 라이브러리들을 가져옵니다.

In [1]:
# HTTP 요청을 위한 라이브러리
import requests

# JSON 데이터 처리를 위한 라이브러리
import json

# CSV 파일 저장을 위한 라이브러리
import csv

# 파일 시스템 작업을 위한 라이브러리
import os

# 지연 시간 처리를 위한 라이브러리
import time

# 타입 힌팅을 위한 라이브러리
from typing import List, Dict, Optional, Any

### 3-2. 설정값 정의 - 하드코딩 필요
API 키, 사용자 ID, 수집할 트윗 수 등 크롤링에 필요한 설정값들을 정의합니다.

In [2]:
# RapidAPI에서 발급받은 API 키 (보안상 실제 키는 환경변수 사용 권장)
API_KEY = "5fac920861msh988e449f8d91b60p10459bjsnba691d3d2d81"

# 수집할 사용자의 [ @이름 / ID(Twitter 사용자 고유 번호) / 설명 ]
'''
@WhiteHouse / 1879644163769335808 / 백악관 = 도널드 트럼프 대통령
@SecScottBessent / 1889019333960998912 / 스콧베센트 재무장관
@JDVance / 1542228578 / 밴스 부통령
@marcorubio / 15745368 / 마르코 루비오 국무장관
@elonmusk / 44196397 / 일론 머스크 테슬라 CEO
@sundarpichai / 14130366 / 순다르 피차이 구글 CEO
@tim_cook / 1636590253 / 팀 쿡 애플 CEO
@CathieDWood / 2361631088 / ARK Invest CEO, 혁신 성장주 투자, 시장 트렌드 주도
@BillAckman / 880412538625810432 / 빌 액먼 펀드매니저, 행동주의 투자 성향
@RayDalio / 62603893 / 브리지워터 창립자, 거시경제 분석, 투자 전략가
@michaelbatnick / 93529573 / 투자 분석, 금융 인사이트 제공
@LizAnnSonders / 2961589380 / 찰스슈왑 수석 투자전략가, 시장 전망, 투자 전략
@Ajay_Bagga / 86437069 / 글로벌 매크로 전문가, 시장 전망, 투자 전략
'''
USER_ID = "86437069"

# 수집할 최대 트윗 수
MAX_TWEETS = 1000

### 3-3. RapidAPI Twitter 크롤러 클래스 정의
트윗 수집 기능을 담은 클래스를 정의합니다. 모든 메서드가 클래스 내부에 포함되어 있습니다.

In [3]:
class RapidAPITweetCrawler:

    # 크롤러 초기화 함수
    def __init__(self, api_key: str):
        if not api_key:
            raise ValueError("API 키가 제공되지 않았습니다.")
            
        # RapidAPI 인증 정보 설정
        self.api_key = api_key
        self.base_url = "https://twitter241.p.rapidapi.com/user-tweets"
        self.headers = {
            "x-rapidapi-key": self.api_key,
            "x-rapidapi-host": "twitter241.p.rapidapi.com"
        }
        
        # API 요청당 트윗 수 설정 (최대 200까지 가능)
        self.count_per_request = 200
        
        # cursor 중복 방지를 위한 캐시
        self.used_cursors = set()

    # API 응답 JSON에서 트윗 데이터를 파싱하는 함수
    def _parse_tweets_from_response(self, response_json: Dict[str, Any]) -> List[Dict[str, str]]:
        tweets_data = []
        
        try:
            # Twitter API의 timeline 구조에서 instructions 리스트를 찾습니다.
            instructions = response_json.get('result', {}).get('timeline', {}).get('instructions', [])
            
            # 'TimelineAddEntries' 타입의 instruction에서 트윗 entries를 찾습니다.
            timeline_entries = []
            for instruction in instructions:
                if instruction.get('type') == 'TimelineAddEntries':
                    timeline_entries = instruction.get('entries', [])
                    break
            
            if not timeline_entries:
                return []

            # 각 entry를 순회하며 트윗 데이터를 추출합니다.
            for entry in timeline_entries:
                # 'TimelineTweet' 타입의 콘텐츠만 처리
                item_content = entry.get('content', {}).get('itemContent', {})
                if item_content and item_content.get('itemType') == 'TimelineTweet':
                    # 트윗 결과 데이터를 가져옵니다.
                    tweet_results = item_content.get('tweet_results', {})
                    result = tweet_results.get('result', {})
                    
                    # legacy 필드에 실제 트윗 데이터가 있습니다.
                    legacy_data = result.get('legacy', {})
                    
                    if legacy_data:
                        # 트윗 생성 시간 추출
                        created_at = legacy_data.get('created_at', 'N/A')
                        full_text = ""
                        
                        # 리트윗(RT)인 경우 원본 트윗의 full_text를 가져옵니다.
                        if 'retweeted_status_result' in legacy_data:
                            # 원본 트윗의 legacy 데이터를 찾습니다.
                            original_tweet_legacy = legacy_data.get('retweeted_status_result', {}).get('result', {}).get('legacy', {})
                            full_text = original_tweet_legacy.get('full_text', '')
                        else:
                            # 일반 트윗은 기존 방식대로 full_text를 가져옵니다.
                            full_text = legacy_data.get('full_text', '')

                        # 줄바꿈 문자를 공백으로 변환하고 양 끝 공백 제거
                        full_text = full_text.replace('\n', ' ').strip()
                        
                        # 추출한 데이터를 리스트에 추가
                        tweets_data.append({
                            'created_at': created_at,
                            'full_text': full_text
                        })
        except (AttributeError, KeyError, IndexError):
            # 데이터 파싱 중 오류 발생 시 빈 리스트 반환
            pass
            
        return tweets_data

    # API 응답에서 다음 페이지를 위한 cursor 값을 찾는 함수
    def _find_next_cursor(self, response_json: Dict[str, Any]) -> Optional[str]:
        try:
            instructions = response_json.get('result', {}).get('timeline', {}).get('instructions', [])
            
            # 모든 instruction 타입에서 cursor 찾기
            all_cursors = []
            
            for instruction in instructions:
                # TimelineAddEntries에서 cursor 찾기
                if instruction.get('type') == 'TimelineAddEntries':
                    entries = instruction.get('entries', [])
                    for entry in entries:
                        content = entry.get('content', {})
                        if content.get('entryType') == 'TimelineTimelineCursor':
                            cursor_value = content.get('value')
                            cursor_type = content.get('cursorType', '')
                            
                            if cursor_value and cursor_value not in self.used_cursors:
                                all_cursors.append({
                                    'value': cursor_value,
                                    'type': cursor_type,
                                    'priority': 1 if cursor_type == 'Bottom' else 2
                                })
                
                # TimelineReplaceEntry에서도 cursor 찾기
                elif instruction.get('type') == 'TimelineReplaceEntry':
                    entry = instruction.get('entry', {})
                    content = entry.get('content', {})
                    if content.get('entryType') == 'TimelineTimelineCursor':
                        cursor_value = content.get('value')
                        cursor_type = content.get('cursorType', '')
                        
                        if cursor_value and cursor_value not in self.used_cursors:
                            all_cursors.append({
                                'value': cursor_value,
                                'type': cursor_type,
                                'priority': 1 if cursor_type == 'Bottom' else 2
                            })
            
            # cursor를 우선순위에 따라 정렬 (Bottom이 우선)
            if all_cursors:
                all_cursors.sort(key=lambda x: x['priority'])
                selected_cursor = all_cursors[0]['value']
                self.used_cursors.add(selected_cursor)
                return selected_cursor
                
        except (AttributeError, KeyError, IndexError):
            pass
            
        return None

    # 트윗 데이터 리스트와 파일 이름을 저장하는 함수    
    def _save_to_csv(self, tweets_list: List[Dict[str, str]], filename: str):

        try:
            with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
                # 'utf-8-sig'는 Excel에서 한글이 깨지지 않도록 BOM을 추가합니다.
                writer = csv.DictWriter(f, fieldnames=['created_at', 'full_text'])
                writer.writeheader()
                writer.writerows(tweets_list)
            print(f"CSV 파일 저장 완료: {filename}")
        except IOError as e:
            print(f"파일 저장 중 오류 발생: {e}")

    # 특정 사용자의 트윗을 수집하여 CSV 파일로 저장하는 함수
    def fetch_user_tweets(self, user_id: str, max_tweets: int = 1000):
        all_tweets = []
        cursor = None
        request_count = 0
        max_requests = 100  # 무한 루프 방지
        consecutive_empty_responses = 0
        
        # cursor 캐시 초기화
        self.used_cursors.clear()
        
        print(f"사용자 ID {user_id}의 트윗 수집을 시작합니다...")
        
        while len(all_tweets) < max_tweets and request_count < max_requests:
            # count를 동적으로 조정 (남은 트윗 수에 따라)
            remaining_tweets = max_tweets - len(all_tweets)
            current_count = min(self.count_per_request, remaining_tweets)
            
            querystring = {
                "user": user_id,
                "count": str(current_count)
            }
            if cursor:
                querystring["cursor"] = cursor
            
            try:
                response = requests.get(self.base_url, headers=self.headers, params=querystring, timeout=45)
                request_count += 1
                
                if response.status_code == 429:  # Rate limit
                    print("API 요청 한도 초과, 60초 대기...")
                    time.sleep(60)
                    continue
                elif response.status_code != 200:
                    if response.status_code >= 500:  # 서버 에러인 경우 재시도
                        print(f"서버 오류 ({response.status_code}), 10초 후 재시도...")
                        time.sleep(10)
                        continue
                    else:
                        print(f"API 요청 실패: HTTP {response.status_code}")
                        break
                    
                data = response.json()
                
                # 트윗 데이터 파싱
                newly_fetched_tweets = self._parse_tweets_from_response(data)
                
                if not newly_fetched_tweets:
                    consecutive_empty_responses += 1
                    
                    if consecutive_empty_responses >= 3:
                        print("연속으로 빈 응답을 받아 수집을 종료합니다.")
                        break
                else:
                    consecutive_empty_responses = 0
                
                all_tweets.extend(newly_fetched_tweets)
                
                # 중복 제거 (created_at + full_text 기준)
                seen = set()
                unique_tweets = []
                for tweet in all_tweets:
                    tweet_key = (tweet['created_at'], tweet['full_text'])
                    if tweet_key not in seen:
                        seen.add(tweet_key)
                        unique_tweets.append(tweet)
                
                all_tweets = unique_tweets
                
                # 다음 cursor 찾기
                next_cursor = self._find_next_cursor(data)
                if not next_cursor or next_cursor == cursor:
                    print("더 이상 수집할 트윗이 없습니다.")
                    break
                
                cursor = next_cursor

                # API rate limit를 고려한 대기 시간 (요청 수에 따라 조정)
                if request_count % 10 == 0:  # 10번째마다 긴 대기
                    wait_time = 5
                else:
                    wait_time = 1
                    
                time.sleep(wait_time)

            except requests.exceptions.Timeout:
                print("요청 타임아웃, 5초 후 재시도...")
                time.sleep(5)
                continue
            except requests.exceptions.RequestException as e:
                print(f"요청 오류: {e}, 10초 후 재시도...")
                time.sleep(10)
                continue
            except json.JSONDecodeError:
                print("JSON 파싱 오류, 5초 후 재시도...")
                time.sleep(5)
                continue

        if all_tweets:
            # f"user_@Ajay_Bagga_tweets.csv"에서 @이름은 하드코딩으로 넣을것
            # 예시1: f"user_@Ajay_Bagga_tweets.csv"
            # 예시2: f"user_@elonmusk_tweets.csv"
            filename = f"user_@Ajay_Bagga_tweets.csv"
            self._save_to_csv(all_tweets, filename)
            print(f"총 {len(all_tweets)}개의 트윗을 수집했습니다.")
        else:
            print("수집된 트윗이 없습니다.")
            
        return all_tweets


### 3-4. 크롤러 인스턴스 생성 및 트윗 수집 실행
설정된 API 키로 크롤러를 생성하고 트윗 수집을 실행합니다.

In [ ]:
# 크롤러 인스턴스 생성
crawler = RapidAPITweetCrawler(api_key=API_KEY)

# 트윗 수집 실행
collected_tweets = crawler.fetch_user_tweets(user_id=USER_ID, max_tweets=MAX_TWEETS)

# 수집 결과 요약 출력
if collected_tweets:
    print(f"총 {len(collected_tweets)}개의 트윗을 수집했습니다.")
    print(f"결과는 user_@Ajay_Bagga_tweets.csv 파일에 저장되었습니다.")
    
    # 처음 3개 트윗 미리보기
    print("\n=== 수집된 트윗 미리보기 (처음 3개) ===")
    for i, tweet in enumerate(collected_tweets[:3], 1):
        print(f"{i}. [{tweet['created_at']}] {tweet['full_text'][:100]}...")
else:
    print("수집된 트윗이 없습니다.")


# 4. RAPID API 기반 트위터(X) 데이터 전처리 및 VADER 감정분석

### 4-1. 필수 라이브러리 Import
감정분석과 데이터 처리에 필요한 라이브러리들을 가져옵니다.

In [7]:
# 데이터 분석 및 조작을 위한 pandas 라이브러리
import pandas as pd

# VADER 감정분석 모델을 위한 NLTK 라이브러리
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# 정규표현식을 사용한 텍스트 전처리를 위한 re 라이브러리
import re

### 4-2. VADER 감정분석 사전 다운로드
VADER 감정분석에 필요한 어휘 사전을 다운로드합니다. (최초 1회만 실행)


In [ ]:
# VADER 감정분석에 필요한 어휘 사전을 다운로드
# vader_lexicon: 감정 점수가 매핑된 단어 사전 데이터
# 최초 한 번만 실행하면 로컬에 저장됨
nltk.download('vader_lexicon')

### 4-3. 다중 사용자 트윗 데이터 병합
X_data 디렉토리의 모든 `user_*.csv` 파일들을 하나로 병합하여 통합 데이터셋을 생성합니다.


In [ ]:
# 파일 시스템 작업에 필요한 추가 라이브러리 import
import glob  # 파일 패턴 매칭을 위한 라이브러리
import os    # 운영체제 인터페이스를 위한 라이브러리
from datetime import datetime  # 날짜/시간 처리를 위한 라이브러리

# glob 패턴을 사용하여 현재 디렉토리에서 "user_"로 시작하고 ".csv"로 끝나는 모든 파일 찾기
csv_files = glob.glob("../data/user_*.csv")
print(f"발견된 모든 CSV 파일: {csv_files}")
print(f"총 {len(csv_files)}개의 파일을 병합합니다.")

# 각 CSV 파일의 데이터프레임을 저장할 리스트 초기화
dfs = []

# 발견된 모든 CSV 파일을 순회하며 데이터 로드
for file in csv_files:
    # 파일이 실제로 존재하는지 확인
    if os.path.exists(file):
        # CSV 파일을 pandas 데이터프레임으로 읽기
        temp_df = pd.read_csv(file)
        
        # 파일명에서 사용자명 추출 (예: "user_@elonmusk_tweets.csv" → "@elonmusk")
        username = file.replace("user_", "").replace("_tweets.csv", "")
        
        # 각 행에 해당 사용자명을 컬럼으로 추가
        temp_df['username'] = username
        
        # 리스트에 데이터프레임 추가
        dfs.append(temp_df)
        print(f"{file} 읽기 완료 - {len(temp_df)}개 행")

# 모든 데이터프레임을 하나로 병합
if dfs:
    # concat 함수를 사용하여 세로로 병합하고 인덱스 리셋
    df = pd.concat(dfs, ignore_index=True)
    print(f"🎉 총 {len(df)}개 행이 병합되었습니다!")
    print(f"📊 포함된 사용자 수: {len(df['username'].unique())}")
else:
    print("읽을 수 있는 파일이 없습니다.")

### 전처리 시작 - VADER 감정분석 전처리를 최소한으로 유지한 이유

- 소셜미디어 특화 설계
- 감정 강화 표현 인식: 대문자, 반복 문자, 이모지 자동 처리
- 구어체 친화적: 축약형, 속어, 인터넷 용어에 강함
- 문맥 인식: 부정어, 강조어, 수식어 조합 이해
- 과도한 전처리의 위험으로 인한 정보 손실 사례 방지


### 4-4. URL 제거 함수 정의

트윗 텍스트에서 URL을 제거하는 전처리 함수를 정의합니다. URL은 감정분석에 불필요한 노이즈입니다.

In [10]:
def remove_urls(text):
    # 정규표현식을 사용하여 HTTP/HTTPS URL 패턴을 찾아서 공백으로 대체
    # r'https?://\S+' 패턴 설명:
    # - https? : http 또는 https (? 는 s가 있거나 없거나)
    # - :// : 프로토콜 구분자
    # - \S+ : 공백이 아닌 문자가 1개 이상 연속 (URL의 나머지 부분)
    cleaned_text = re.sub(r'https?://\S+', '', text)
    
    # 앞뒤 공백 제거하여 반환
    return cleaned_text.strip()

### 4-5. 트윗 텍스트에서 URL 제거 적용
정의한 URL 제거 함수를 모든 트윗 텍스트에 적용하여 데이터를 정제합니다.

In [11]:
# pandas의 apply() 함수를 사용하여 모든 트윗 텍스트에 URL 제거 함수 적용
# apply()는 시리즈의 각 요소에 함수를 적용하여 새로운 시리즈를 반환
df['full_text'] = df['full_text'].apply(remove_urls)

### 4-6. 빈 문자열을 NaN으로 변환
URL 제거 후 빈 문자열이 된 텍스트들을 pandas의 NaN(결측값)으로 변환합니다.


In [ ]:
# 빈 문자열('')을 pandas의 NA(Not Available) 값으로 변환
# replace() 함수: 첫 번째 인자를 두 번째 인자로 대체
# inplace=True: 원본 데이터프레임을 직접 수정 (새로운 객체 생성 안 함)
# pd.NA: pandas 2.0+에서 권장하는 결측값 표현
df['full_text'].replace('', pd.NA, inplace=True)

### 4-7. 결측값(NaN) 제거
감정분석이 불가능한 빈 텍스트나 결측값을 가진 행들을 데이터셋에서 제거합니다.

In [12]:
# dropna() 함수를 사용하여 결측값을 가진 행들을 제거
# subset=['full_text']: full_text 컬럼에서 NaN 값을 가진 행들만 제거
# inplace=True: 원본 데이터프레임을 직접 수정
# 감정분석을 위해서는 텍스트 내용이 필수이므로 빈 텍스트는 제거 필요
df.dropna(subset=['full_text'], inplace=True)

### 4-8. 날짜 형식 변환 및 데이터 정렬
트위터 API 형식의 날짜를 표준 날짜 형식으로 변환하고, 최신 순으로 정렬합니다.

In [ ]:
def convert_date_format(date_str):

    try:
        # 트위터 API 날짜 형식: "Mon Jun 16 02:50:54 +0000 2025"
        # pd.to_datetime으로 파싱하고 strftime으로 원하는 형식으로 변환
        # format 매개변수 설명:
        # %a: 축약된 요일명 (Mon, Tue, ...)
        # %b: 축약된 월명 (Jan, Feb, ...)  
        # %d: 일 (01-31)
        # %H:%M:%S: 시:분:초
        # %z: 타임존 오프셋 (+0000)
        # %Y: 4자리 연도
        dt = pd.to_datetime(date_str, format='%a %b %d %H:%M:%S %z %Y')
        
        # 타임존 제거하고 "YYYY-MM-DD HH:MM:SS" 형식으로 변환
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except:
        # 파싱 실패 시 원본 그대로 반환
        return date_str

# 날짜 형식 변환 프로세스 시작
print("날짜 형식 변환 중...")

# 모든 트윗의 created_at 컬럼에 날짜 변환 함수 적용
df['created_at'] = df['created_at'].apply(convert_date_format)

# 문자열을 pandas datetime 객체로 변환 (정렬 및 시간 연산을 위해)
df['created_at'] = pd.to_datetime(df['created_at'])

# 최신 트윗이 맨 위에 오도록 내림차순 정렬
# ascending=False: 내림차순 (큰 값부터 작은 값 순)
# reset_index(drop=True): 정렬 후 인덱스를 0부터 다시 시작
df = df.sort_values(by='created_at', ascending=False).reset_index(drop=True)

print("날짜 형식 변환 및 정렬 완료!")
print(f"날짜 범위: {df['created_at'].min()} ~ {df['created_at'].max()}")
print("\n결과 확인:")
print(df.head())

### 4-9. VADER 감정분석기 초기화
VADER(Valence Aware Dictionary and sEntiment Reasoner) 감정분석 모델을 초기화합니다.

In [14]:
# VADER 감정분석기 인스턴스 생성
# SentimentIntensityAnalyzer: VADER 알고리즘을 구현한 클래스
# - 소셜 미디어 텍스트에 특화된 감정분석 도구
# - 이모티콘, 대문자, 구두점, 단어 조합 등을 고려하여 감정 점수 계산
# - positive, negative, neutral, compound 점수를 반환
sia = SentimentIntensityAnalyzer()

### 4-10. 감정분석 함수 정의
텍스트를 분석하여 positive/negative/neutral로 분류하고 각 감정 점수를 반환하는 함수를 정의합니다.

In [15]:
def analyze_sentiment(text):
    # VADER 감정분석기로 텍스트 분석
    # polarity_scores() 반환값:
    # - 'neg': negative 감정 점수 (0~1)
    # - 'neu': neutral 감정 점수 (0~1)  
    # - 'pos': positive 감정 점수 (0~1)
    # - 'compound': 복합 점수 (-1~1, 전체적인 감정 강도)
    scores = sia.polarity_scores(text)
    
    # compound 점수를 기준으로 감정 분류
    compound = scores['compound']
    
    # VADER 권장 임계값을 사용한 감정 분류
    if compound >= 0.05:
        # compound >= 0.05: 긍정적 감정
        sentiment = 'positive'
    elif compound <= -0.05:
        # compound <= -0.05: 부정적 감정
        sentiment = 'negative'
    else:
        # -0.05 < compound < 0.05: 중립적 감정
        sentiment = 'neutral'
    
    # pandas Series로 반환 (DataFrame의 새 컬럼들로 할당하기 위함)
    return pd.Series([sentiment, scores['neg'], scores['neu'], scores['pos']])

### 4-11. 모든 트윗에 감정분석 실행
정의한 감정분석 함수를 모든 트윗 텍스트에 적용하여 감정 컬럼들을 생성합니다.

In [16]:
# pandas의 apply() 함수를 사용하여 모든 트윗에 감정분석 함수 적용
# analyze_sentiment() 함수가 pd.Series를 반환하므로 
# 여러 컬럼에 동시에 할당 가능
# - sentiment: 감정 분류 라벨 ('positive', 'negative', 'neutral')
# - neg: 부정 감정 점수 (0~1)
# - neu: 중립 감정 점수 (0~1)
# - pos: 긍정 감정 점수 (0~1)
df[['sentiment', 'neg', 'neu', 'pos']] = df['full_text'].apply(analyze_sentiment)

### 4-12. 최종 데이터 컬럼 선택
분석에 필요한 핵심 컬럼들만 선택하여 최종 데이터셋을 구성합니다.

In [17]:
# 최종 분석 결과에 필요한 컬럼들만 선택
# - created_at: 트윗 작성 날짜/시간
# - full_text: 전처리된 트윗 텍스트 (URL 제거됨)
# - username: 트윗 작성자 (@사용자명)
# - sentiment: 감정 분류 결과 (positive/negative/neutral)
# - neg: 부정 감정 점수
# - neu: 중립 감정 점수  
# - pos: 긍정 감정 점수
df = df[['created_at', 'full_text', 'username', 'sentiment', 'neg', 'neu', 'pos']]

### 4-13. 최종 결과 저장 및 요약
감정분석이 완료된 트윗 데이터를 CSV 파일로 저장하고 처리 결과를 요약합니다.

In [ ]:
# 결과 파일 저장
output_filename = "merged_tweets_with_sentiment.csv"

# 감정분석이 완료된 데이터프레임을 CSV 파일로 저장
# index=False: 행 인덱스를 파일에 포함하지 않음
df.to_csv(output_filename, index=False)

# 처리 결과 요약 정보 출력
print(f"✅ 결과가 {output_filename}에 저장되었습니다.")
print(f"📊 총 {len(df)}개의 트윗이 처리되었습니다.")

# 감정분석에 포함된 사용자 목록 (알파벳 순 정렬)
print(f"👥 포함된 사용자: {sorted(df['username'].unique())}")

# 트윗 데이터의 시간 범위
print(f"📅 날짜 범위: {df['created_at'].min()} ~ {df['created_at'].max()}")

# 감정 분류별 트윗 수 통계
print(f"\n📈 감정 분류별 통계:")
sentiment_counts = df['sentiment'].value_counts()
for sentiment, count in sentiment_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  - {sentiment}: {count}개 ({percentage:.1f}%)")

# 사용자별 트윗 수 통계 (상위 5명)
print(f"\n🏆 사용자별 트윗 수 (상위 5명):")
user_counts = df['username'].value_counts().head(5)
for username, count in user_counts.items():
    print(f"  - {username}: {count}개")

# 5. Y finance API 기반 주가 데이터 수집

### 5-1. 필수 라이브러리 Import

In [31]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
import numpy as np
warnings.filterwarnings('ignore')


### 5-2. RSI 계산 함수

In [32]:
def calculate_rsi(prices, window=14):
    
    # 전일 대비 가격 변화량 계산
    delta = prices.diff()
    
    # 상승일의 가격 상승폭만 추출 (하락일은 0으로 처리)
    # where 조건: delta > 0인 경우만 해당 값 사용, 나머지는 0
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    
    # 하락일의 가격 하락폭만 추출 (상승일은 0으로 처리)
    # -delta로 하락폭을 양수로 변환
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    
    # RS (Relative Strength) = 평균 상승폭 / 평균 하락폭
    rs = gain / loss
    
    # RSI = 100 - (100 / (1 + RS))
    # RSI가 70 이상이면 과매수, 30 이하면 과매도 구간으로 해석
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


### 5-3. 시간 조정 함수

In [33]:
def adjust_time_to_hour(df):
    
    # Datetime 컬럼이 존재하는지 확인
    if 'Datetime' in df.columns:
        # 문자열을 pandas datetime 객체로 변환
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        
        # 시간을 시간 단위로 내림 (분, 초를 0으로 만듦)
        # floor('H'): 시간 단위로 내림 처리
        # 예: 2024-01-15 13:30:45 → 2024-01-15 13:00:00
        df['Datetime'] = df['Datetime'].dt.floor('H')
        
        # 동일한 시간이 여러 개 있는 경우 마지막 값만 유지
        # keep='last': 중복된 값 중 마지막 행을 유지
        df = df.drop_duplicates(subset=['Datetime'], keep='last')
        
    return df


### 5-4. 기술적 지표 추가 함수 (1시간 간격용)

In [34]:
def add_technical_features(df):
    
    # === 수익률 계산 ===
    # 전 시간 대비 수익률 계산 (Close[t] - Close[t-1]) / Close[t-1]
    df['Returns'] = df['Close'].pct_change()
    
    # 로그 수익률 계산 ln(Close[t] / Close[t-1])
    # 연속복리 개념으로, 작은 변화에서는 일반 수익률과 유사
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # === 이동평균 (Simple Moving Average) ===
    # 단순 이동평균: 지정 기간의 평균 가격
    df['SMA_10'] = df['Close'].rolling(window=10).mean()  # 10시간 평균
    df['SMA_20'] = df['Close'].rolling(window=20).mean()  # 20시간 평균
    df['SMA_50'] = df['Close'].rolling(window=50).mean()  # 50시간 평균
    
    # === 지수이동평균 (Exponential Moving Average) ===
    # 최근 데이터에 더 높은 가중치를 부여하는 이동평균
    df['EMA_12'] = df['Close'].ewm(span=12).mean()  # 12시간 EMA
    df['EMA_26'] = df['Close'].ewm(span=26).mean()  # 26시간 EMA
    
    # === MACD (Moving Average Convergence Divergence) ===
    # 단기 EMA와 장기 EMA의 차이로 추세 변화를 파악
    df['MACD'] = df['EMA_12'] - df['EMA_26']  # MACD 라인
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()  # 시그널 라인 (9시간 EMA)
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']  # 히스토그램 (매수/매도 신호)
    
    # === RSI (Relative Strength Index) ===
    # 과매수/과매도 상태를 나타내는 모멘텀 지표 (0~100)
    df['RSI'] = calculate_rsi(df['Close'])
    
    # === 볼린저 밴드 (Bollinger Bands) ===
    # 가격의 변동성을 기반으로 한 기술적 지표
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()  # 중심선 (20시간 이동평균)
    bb_std = df['Close'].rolling(window=20).std()  # 20시간 표준편차
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)  # 상단 밴드 (평균 + 2*표준편차)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)  # 하단 밴드 (평균 - 2*표준편차)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']  # 밴드 폭 (변동성 지표)
    # 현재 가격이 밴드 내에서 어느 위치에 있는지 (0~1, 0.5가 중앙)
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # === 변동성 지표 ===
    # 수익률의 표준편차로 가격 변동성 측정
    df['Volatility_10'] = df['Returns'].rolling(window=10).std()  # 10시간 변동성
    df['Volatility_20'] = df['Returns'].rolling(window=20).std()  # 20시간 변동성
    
    # === 가격 변화 지표 ===
    # 시가 대비 종가 변화 (절대값)
    df['Price_Change'] = df['Close'] - df['Open']
    
    # 시가 대비 종가 변화 (백분율)
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # === High-Low 스프레드 ===
    # 당일 최고가와 최저가의 차이 (일중 변동폭)
    df['HL_Spread'] = df['High'] - df['Low']
    
    # 종가 대비 일중 변동폭 비율
    df['HL_Spread_Pct'] = (df['High'] - df['Low']) / df['Close'] * 100
    
    # === 시간 특성 ===
    # 시간대별 패턴 분석을 위한 특성
    df['Hour'] = df['Datetime'].dt.hour  # 시간 (0~23)
    df['DayOfWeek'] = df['Datetime'].dt.dayofweek  # 요일 (0=월요일, 6=일요일)
    df['Month'] = df['Datetime'].dt.month  # 월 (1~12)
    df['Quarter'] = df['Datetime'].dt.quarter  # 분기 (1~4)
    
    # === 거래시간 분류 ===
    # 미국 주식시장 기준 거래시간 분류
    df['Is_Trading_Hours'] = ((df['Hour'] >= 9) & (df['Hour'] <= 16)).astype(int)  # 정규 거래시간
    df['Is_Market_Open'] = ((df['Hour'] >= 9) & (df['Hour'] < 16)).astype(int)     # 시장 개장시간
    df['Is_Premarket'] = ((df['Hour'] >= 4) & (df['Hour'] < 9)).astype(int)       # 프리마켓 (4:00-9:30)
    df['Is_Aftermarket'] = ((df['Hour'] >= 16) & (df['Hour'] <= 20)).astype(int)  # 애프터마켓 (16:00-20:00)
    df['Is_Extended_Hours'] = (df['Is_Premarket'] | df['Is_Aftermarket']).astype(int)  # 연장거래시간
    
    return df


### 5-5. 1시간 간격 주식 데이터 수집 함수

In [35]:
def get_hourly_stock_data(ticker, days=365, save_to_csv=True):
    
    try:
        # yfinance 1시간 간격 데이터 제약사항 확인 (최대 730일)
        if days > 730:
            days = 730
        
        # 현재 날짜 기준으로 시작일과 종료일 계산
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        # yfinance를 사용하여 주식 데이터 다운로드
        stock_data = yf.download(
            ticker,  # 티커 심볼
            start=start_date.strftime('%Y-%m-%d'),  # 시작일 (YYYY-MM-DD 형식)
            end=end_date.strftime('%Y-%m-%d'),      # 종료일 (YYYY-MM-DD 형식)
            interval='1h',    # 1시간 간격
            prepost=True,     # 시장 외 시간 데이터 포함 (프리마켓, 애프터마켓)
            progress=False    # 진행상황 표시 안 함
        )
        
        # 데이터가 비어있는 경우 None 반환
        if stock_data.empty:
            return None
        
        # 인덱스(날짜/시간)를 일반 컬럼으로 변환
        stock_data = stock_data.reset_index()
        
        # === 컬럼명 정리 작업 ===
        # yfinance의 다양한 반환 형식에 대응
        
        # 'Date' 컬럼이 있으면 'Datetime'으로 변경
        if 'Date' in stock_data.columns:
            stock_data = stock_data.rename(columns={'Date': 'Datetime'})
        # 첫 번째 컬럼이 시간 데이터인 경우 'Datetime'으로 변경
        elif stock_data.columns[0] not in ['Datetime', 'Date']:
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # === 멀티레벨 컬럼 처리 ===
        # yfinance가 때로 계층적 컬럼 구조로 데이터를 반환하는 경우
        if isinstance(stock_data.columns, pd.MultiIndex):
            new_columns = []
            for col in stock_data.columns:
                if isinstance(col, tuple):
                    # 튜플의 첫 번째 요소가 시간 관련이면 'Datetime'으로
                    if col[0] == 'Datetime' or 'Date' in str(col[0]):
                        new_columns.append('Datetime')
                    else:
                        # 그 외는 첫 번째 요소만 사용 (Open, High, Low, Close, Volume)
                        new_columns.append(col[0])
                else:
                    new_columns.append(col)
            stock_data.columns = new_columns
        
        # 최종적으로 'Datetime' 컬럼이 없으면 첫 번째 컬럼을 사용
        if 'Datetime' not in stock_data.columns:
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # === 데이터 전처리 및 특성 추가 ===
        # 시간을 정시로 조정 (예: 13:30 → 13:00)
        stock_data = adjust_time_to_hour(stock_data)
        
        # 기술적 지표 계산 및 추가
        stock_data = add_technical_features(stock_data)
        
        # === CSV 파일 저장 ===
        if save_to_csv:
            # 파일명 형식: {TICKER}_1hour_data_{DAYS}days.csv
            filename = f"{ticker}_1hour_data_{days}days.csv"
            stock_data.to_csv(filename, index=False)  # 인덱스 제외하고 저장
        
        return stock_data
        
    except Exception as e:
        # 에러 발생 시 None 반환
        return None


### 5-6. 다중 티커 1시간 데이터 수집 함수

In [36]:
def get_multiple_tickers_hourly(tickers, days=365, save_individual=True, save_combined=True):
    
    # 수집된 데이터를 저장할 딕셔너리 초기화
    all_data = {}
    
    # 각 티커에 대해 순차적으로 데이터 수집
    for ticker in tickers:
        # 개별 티커 데이터 수집 (save_to_csv는 save_individual 설정에 따라)
        data = get_hourly_stock_data(ticker, days=days, save_to_csv=save_individual)
        
        # 수집된 데이터가 유효한 경우 딕셔너리에 저장
        if data is not None:
            all_data[ticker] = data
    
    # === 통합 데이터 파일 생성 ===
    if save_combined and all_data:
        # 모든 티커 데이터를 하나로 합칠 빈 데이터프레임 생성
        combined_data = pd.DataFrame()
        
        # 각 티커 데이터에 티커 컬럼 추가 후 통합
        for ticker, data in all_data.items():
            # 원본 데이터를 복사하여 수정
            ticker_data = data.copy()
            
            # 티커 식별을 위한 'Ticker' 컬럼 추가
            ticker_data['Ticker'] = ticker
            
            # 기존 통합 데이터에 현재 티커 데이터 추가
            # ignore_index=True: 인덱스를 새로 생성 (연속적인 번호)
            combined_data = pd.concat([combined_data, ticker_data], ignore_index=True)
        
        # 통합 데이터 CSV 파일 저장
        combined_filename = f"multiple_stocks_1hour_data_{days}days.csv"
        combined_data.to_csv(combined_filename, index=False)
    
    # 티커별 데이터가 담긴 딕셔너리 반환
    return all_data


### 5-7. 데이터 요약 분석 함수

In [37]:
def analyze_data_summary(data_dict):
    
    # 딕셔너리의 각 티커와 데이터에 대해 반복
    for ticker, data in data_dict.items():
        # 데이터가 유효한 경우에만 분석 진행
        if data is not None:
            # 전체 데이터에서 결측치(NaN) 개수 계산
            # isnull(): 각 셀이 결측치인지 True/False 반환
            # sum().sum(): 먼저 각 컬럼별 결측치 개수를 구한 후, 전체 합계 계산
            missing_count = data.isnull().sum().sum()
            
            # 결과 출력 (데이터 포인트 개수와 결측치 개수)
            # len(data): 총 행(데이터 포인트) 개수
            # :,: 천 단위 구분자 추가 (예: 1000 → 1,000)
            print(f"{ticker}: {len(data):,}개 포인트, 결측치: {missing_count}개")


### 5-8. 주요 종목 1시간 데이터 수집 실행

In [ ]:
# === 수집 대상 주요 종목 설정 ===
# 대형주 기술주 중심으로 선정 (시가총액, 거래량, 변동성 고려)
# - AAPL: 애플 (아이폰, 맥북 등 하드웨어)
# - AMZN: 아마존 (전자상거래, 클라우드 서비스)
# - TSLA: 테슬라 (전기자동차, 에너지)
# - GOOGL: 구글 (검색엔진, 광고, 클라우드)
# - MSFT: 마이크로소프트 (윈도우, 오피스, 클라우드)
tickers = ['AAPL', 'AMZN', 'TSLA', 'GOOGL', 'MSFT']

# === 1시간 간격 데이터 수집 실행 ===
# days=365: 1년간의 데이터 수집 (충분한 학습 데이터 확보)
# save_individual=True: 각 종목별 개별 CSV 파일 생성
# save_combined=True: 모든 종목을 통합한 CSV 파일도 생성
all_stock_data = get_multiple_tickers_hourly(tickers, days=365)

# === 수집 결과 요약 분석 ===
# 각 종목별 데이터 포인트 개수와 결측치 개수 확인
analyze_data_summary(all_stock_data)

# 수집 완료된 종목 개수 출력
print(f"\\n데이터 수집 완료: {len(all_stock_data)}개 종목")


### 5-9. AAPL 데이터 상세 분석

In [ ]:
# === AAPL 데이터 존재 여부 확인 및 상세 분석 ===
if 'AAPL' in all_stock_data:
    # AAPL 1시간 데이터를 변수에 저장
    aapl_1h = all_stock_data['AAPL']
    
    # === 기본 데이터 미리보기 ===
    print("AAPL 1시간 데이터 미리보기:")
    # 핵심 OHLCV 컬럼만 선택하여 상위 5개 행 출력
    # OHLCV: Open(시가), High(고가), Low(저가), Close(종가), Volume(거래량)
    print(aapl_1h[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']].head())
    
    # === 거래시간 데이터 필터링 ===
    # LSTM 학습에는 정규 거래시간 데이터가 더 안정적
    # Is_Trading_Hours == 1: 미국 시장 정규 거래시간 (9:30-16:00 ET)
    trading_hours = aapl_1h[aapl_1h['Is_Trading_Hours'] == 1]
    
    # === 학습용 데이터 분석 ===
    print(f"\\n학습용 데이터 분석:")
    print(f"전체 시간 개수: {len(aapl_1h):,}개")  # 프리마켓, 애프터마켓 포함
    print(f"거래시간 개수: {len(trading_hours):,}개")  # 정규 거래시간만
    
    # === 기술적 지표 컬럼 확인 ===
    # LSTM 모델의 입력 특성으로 사용할 주요 기술적 지표들
    tech_indicators = [col for col in aapl_1h.columns if col in ['SMA_10', 'SMA_20', 'RSI', 'MACD', 'BB_Upper', 'BB_Lower']]
    print(f"기술적 지표 컬럼: {tech_indicators}")
    
    # === 데이터 품질 분석 ===
    # 결측치가 있는 기술적 지표 확인
    missing_indicators = aapl_1h[tech_indicators].isnull().sum()
    print(f"\\n기술적 지표별 결측치:")
    for indicator, missing_count in missing_indicators.items():
        if missing_count > 0:
            print(f"  {indicator}: {missing_count}개")
    
    # === 데이터 범위 정보 ===
    print(f"\\n데이터 기간:")
    print(f"시작: {aapl_1h['Datetime'].min()}")
    print(f"종료: {aapl_1h['Datetime'].max()}")
    print(f"총 기간: {(aapl_1h['Datetime'].max() - aapl_1h['Datetime'].min()).days}일")


### 5-10. 데이터 수집 완료 요약

In [ ]:
# === 데이터 수집 완료 최종 요약 ===
print("\\n 주식 데이터 수집 완료!")

# 성공적으로 수집된 종목 목록 출력
print(f"수집된 종목: {list(all_stock_data.keys())}")

# === 생성된 CSV 파일 목록 확인 ===
print("\\n 생성된 CSV 파일:")

# 현재 디렉토리의 파일 목록을 가져와서 CSV 파일만 필터링
import os
# 리스트 컴프리헨션으로 CSV 파일 중 수집 대상 티커가 포함된 파일만 선택
csv_files = [f for f in os.listdir('.') if f.endswith('.csv') and any(ticker in f for ticker in tickers)]

# 각 생성된 CSV 파일을 출력
for file in csv_files:
    print(f"  {file}")

# 6. 트윗 - 주가 데이터 병합 및 라벨링

In [ ]:
# =============================================================================
# 트윗-주가 데이터 병합 및 라벨링 (시계열 특성 추출)
# =============================================================================

import pandas as pd  # 데이터 조작 및 분석을 위한 pandas 라이브러리 임포트

# 파일 경로 설정
stock_path = "./AAPL_1hour_data_365days.csv"  # AAPL 1시간 단위 주가 데이터 파일 경로 (동일)
tweets_path = "./merged_tweets_with_sentiment.csv"  # VADER 감정분석이 완료된 트윗 데이터 파일 경로

# 데이터 불러오기 (날짜 컬럼을 자동으로 datetime 타입으로 변환)
stock_df = pd.read_csv(stock_path, parse_dates=["Datetime"])  # 주가 데이터 로드, Datetime 컬럼을 날짜 타입으로 파싱
tweets_df = pd.read_csv(tweets_path, parse_dates=["created_at"])  # 트윗 데이터 로드, created_at 컬럼을 날짜 타입으로 파싱

# 타임존 제거 (서로 다른 타임존으로 인한 병합 문제 해결)
stock_df["Datetime"] = stock_df["Datetime"].dt.tz_localize(None)  # 주가 데이터의 타임존 정보 제거 (UTC나 로컬 타임존 무관하게 만듦)
tweets_df["created_at"] = tweets_df["created_at"].dt.tz_localize(None)  # 트윗 데이터의 타임존 정보 제거

# 시계열 데이터 정렬 (시간 순서대로 정렬하여 과거/미래 탐색을 정확하게 함)
stock_df = stock_df.sort_values("Datetime").reset_index(drop=True)  # 날짜 순으로 정렬하고 인덱스 재설정
tweets_df = tweets_df.sort_values("created_at").reset_index(drop=True)  # 트윗도 시간순으로 정렬

# 불필요한 거래 시간 관련 컬럼 제거 (모델 학습에 불필요한 boolean 변수들 제거)
exclude_cols = ['Is_Trading_Hours', 'Is_Market_Open', 'Is_Premarket', 'Is_Aftermarket', 'Is_Extended_Hours']  # 제거할 컬럼 리스트
stock_df = stock_df.drop(columns=[col for col in exclude_cols if col in stock_df.columns])  # 해당 컬럼들이 존재하면 제거

# 병합 결과를 저장할 빈 리스트 초기화
tweet_rows = []  # 각 트윗에 대한 특성과 라벨을 담을 딕셔너리들의 리스트

# 각 트윗별로 반복 처리 (트윗 발생 시점 기준으로 주가 변화 패턴 추출)
for idx, tweet_row in tweets_df.iterrows():  # 트윗 데이터프레임의 각 행을 순회
    tweet_time = tweet_row['created_at']  # 현재 트윗의 생성 시간 추출
    
    if idx % 1000 == 0:  # 진행상황 출력 (1000개마다)
        print(f"처리 중: {idx}/{len(tweets_df)} 트윗")

    # 트윗 발행 이후 가장 가까운 미래 주가 데이터 찾기 (예측 타겟)
    future_stock = stock_df[stock_df['Datetime'] > tweet_time].head(1)  # 트윗 시간 이후의 첫 번째 주가 데이터 선택
    if future_stock.empty:  # 미래 주가 데이터가 없으면 (트윗이 가장 최근인 경우)
        continue  # 해당 트윗은 건너뛰고 다음 트윗으로 진행

    # 예측 타겟이 될 주가 정보 추출
    target_row = future_stock.iloc[0]  # 미래 주가 데이터의 첫 번째 행
    target_time = target_row['Datetime']  # 타겟 주가의 시간
    target_close = target_row['Close']  # 타겟 주가의 종가 (예측해야 할 값)

    # 타겟 시점 이전의 과거 3개 주가 데이터 추출 (시계열 특성으로 사용)
    past_rows = stock_df[stock_df['Datetime'] < target_time].tail(3)  # 타겟 시간 이전의 마지막 3개 주가 데이터
    if len(past_rows) < 3:  # 충분한 과거 데이터가 없으면 (데이터 초기 시점)
        continue  # 해당 트윗은 건너뛰고 다음 트윗으로 진행

    # 수익률 계산을 위한 기준점 (과거 데이터 중 가장 최근 종가)
    past_last_close = past_rows.iloc[-1]['Close']  # 과거 3개 데이터 중 가장 최근 종가

    # 수익률 계산 및 분류 라벨 생성
    return_pct = (target_close - past_last_close) / past_last_close * 100  # 백분율 수익률 계산
    # 3클래스 분류: 0.4% 이상 상승(1), -0.4% 이하 하락(-1), 그 외 보합(0)
    label = 1 if return_pct >= 0.4 else (-1 if return_pct <= -0.4 else 0)  # 임계값 기반 라벨링

    # 한 행의 데이터를 담을 딕셔너리 생성 (특성 + 라벨)
    row = {
        "tweet_id": idx,  # 트윗 인덱스 (고유 ID 역할)
        "tweet_time": tweet_time,  # 트윗 생성 시간
        "target_close": target_close,  # 예측 타겟 종가
        "target_return_pct": return_pct,  # 실제 수익률 (연속값)
        "target_multi_raw": label,  # 원본 분류 라벨 (-1, 0, 1)
        "vader_positive": tweet_row['pos'],  # VADER 긍정 감정 점수
        "vader_neutral": tweet_row['neu'],  # VADER 중립 감정 점수
        "vader_negative": tweet_row['neg'],  # VADER 부정 감정 점수
    }

    # 과거 3개 주가 데이터를 평면화(flatten)하여 특성으로 추가
    for i, (_, stock_row) in enumerate(past_rows.iterrows(), 1):  # 과거 3개 데이터를 순회 (1부터 시작하는 인덱스)
        for col in stock_df.columns:  # 주가 데이터의 모든 컬럼에 대해
            if col == "Datetime":  # 날짜 컬럼은 제외
                continue
            # x1_Open, x1_High, x2_Open, x2_High, x3_Open, x3_High 등의 형태로 특성 생성
            row[f"x{i}_{col}"] = stock_row[col]  # i번째 과거 데이터의 각 컬럼값을 특성으로 추가

    tweet_rows.append(row)  # 완성된 행 데이터를 리스트에 추가

# 모든 행 데이터를 하나의 DataFrame으로 변환
tweet_merged_df = pd.DataFrame(tweet_rows)  # 리스트의 딕셔너리들을 DataFrame으로 변환

# 감정 라벨을 수치형으로 변환 (머신러닝 알고리즘을 위해)
# sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}  # 감정 라벨을 숫자로 매핑
# tweet_merged_df["sentiment_numeric"] = tweet_merged_df["sentiment_label"].map(sentiment_map)  # 수치형 감정 라벨 생성

# 머신러닝 알고리즘을 위한 타겟 라벨 변환 (음수 라벨을 0부터 시작하도록 변경)
label_map = {-1: 0, 0: 1, 1: 2}  # -1(하락)→0, 0(보합)→1, 1(상승)→2로 매핑
tweet_merged_df["target_multi"] = tweet_merged_df["target_multi_raw"].map(label_map)  # 새로운 라벨 컬럼 생성

# 최종 결과를 CSV 파일로 저장
tweet_merged_df.to_csv("tweet_stock_classification.csv", index=False)  # 인덱스 없이 CSV로 저장

# 결과 요약 정보 출력
print("병합 완료: tweet_stock_classification.csv 저장됨") # 완료 메시지 출력
